In [4]:
import os
import json
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

VECTOR_STORE_PATH = os.getenv("VECTOR_STORE_PATH")
EMBEDDING_MODEL_NAME = os.getenv("EMBEDDING_MODEL_NAME")

if not VECTOR_STORE_PATH:
    raise ValueError("VECTOR_STORE_PATH 환경 변수가 설정되지 않았습니다.")
if not EMBEDDING_MODEL_NAME:
    raise ValueError("EMBEDDING_MODEL_NAME 환경 변수가 설정되지 않았습니다.")

# 기존 벡터스토어 확인
if not os.path.exists(VECTOR_STORE_PATH):
    raise FileNotFoundError(f"벡터스토어가 존재하지 않습니다: {VECTOR_STORE_PATH}")

# 임베딩 모델 로드
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_NAME)
print(f"임베딩 모델 '{EMBEDDING_MODEL_NAME}' 로드 완료")

# 벡터스토어 로드
vectorstore = Chroma(
    persist_directory=VECTOR_STORE_PATH,
    embedding_function=embeddings,
)
print(f"벡터스토어 로드 완료: {VECTOR_STORE_PATH}")

def summarize_document(doc):
    try:
        content_json = json.loads(doc.page_content)
        content_list = content_json.get("본문", {}).get("AdmRulService", {}).get("개정문", {}).get("개정문내용", [])
        content_summary = " ".join(content_list) if isinstance(content_list, list) else "요약 불가"
    except json.JSONDecodeError:
        content_summary = doc.page_content[:300] if doc.page_content else "내용 없음"

    metadata = doc.metadata
    return {
        'Key Path': metadata.get('key_path', '알 수 없음'),
        'Source': metadata.get('source', '알 수 없음'),
        'Law Name': metadata.get('법령명', '알 수 없음'),
        'Effective Date': metadata.get('시행일자', '알 수 없음'),
        'Responsible Department': metadata.get('소관부처명', '알 수 없음'),
        'Content Summary': content_summary[:300],
    }
def remove_duplicates(docs):
    seen = set()
    unique_docs = []
    for doc in docs:
        if doc.page_content not in seen:
            unique_docs.append(doc)
            seen.add(doc.page_content)
    return unique_docs

# 검색 결과 출력
def display_search_results(query, vectorstore, top_k=10):
    print(f"\n샘플 쿼리: {query}")
    
    # 벡터스토어에 저장된 문서 수 가져오기
    num_documents = vectorstore._collection.count()  # Chroma 내부 collection의 문서 수 확인
    print(f"벡터스토어에 저장된 문서 수: {num_documents}")
    
    if num_documents == 0:
        print("벡터스토어에 문서가 없습니다.")
        return

    retrieved_docs = remove_duplicates(vectorstore.similarity_search(query, k=min(top_k, num_documents)))
    print(f"검색된 문서 수: {len(retrieved_docs)}")

    if not retrieved_docs:
        print("검색된 문서가 없습니다.")
        return

    for idx, doc in enumerate(retrieved_docs, 1):
        summary = summarize_document(doc)
        print(f"\n  문서 {idx}:")
        for key, value in summary.items():
            print(f"    - {key}: {value if value else '정보 없음'}")


# 샘플 쿼리 실행
sample_queries = [
    "가장 최근에 시행된 법령을 가져오세요.",
    "건축물의 차음 구조에 대한 법령은 무엇인가요?",
    "환경 규제와 관련된 주요 법령을 알려주세요.",
    "공공 건축물 기준에 대한 최근 변경 사항은?"
]

# 기존 벡터스토어 상태 확인 및 검색 테스트 실행
print("\n벡터스토어 상태 확인 및 테스트 실행 시작...")
for query in sample_queries:
    display_search_results(query, vectorstore)
print("\n테스트 실행 완료.")

임베딩 모델 'jhgan/ko-sroberta-nli' 로드 완료
벡터스토어 로드 완료: /Users/joguk/Desktop/law_vector

벡터스토어 상태 확인 및 테스트 실행 시작...

샘플 쿼리: 가장 최근에 시행된 법령을 가져오세요.
벡터스토어에 저장된 문서 수: 3
검색된 문서 수: 3


TypeError: sequence item 0: expected str instance, list found